In [1]:
# Dependencies
import os
import pandas as pd
from sqlalchemy import create_engine
import tqdm

batter_preprocessing should be good - it should have game-level performances from each batter: trust but verify

need to finish pitcher_preprocessing to get this at the same level - one row per starting pitcher + bullpen performance same summation method as batter

In [2]:
# Database connection parameters
db_user = "postgres"
db_password = "1789"
db_name = "baseball"
db_host = "localhost"
db_port = "5433"


In [3]:
# Bring in the data from the postgres database
engine = create_engine(
    f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")


should pitcher columns be whole game? or row specific?

In [4]:
# Grab the pitcher data from the database
pitcher_df = pd.read_sql_table('pitchers', engine)

batter_df = pd.read_sql_table('batter_statistics', engine)

In [5]:
# Redefine the inning column in pitcher_df as a string
pitcher_df['inning'] = pitcher_df['inning'].astype(str)


In [6]:
# Keep only the necessary columns from the batter_df DataFrame
batter_df = batter_df[[
    'game_id', 'home_team', 'away_team', 'inning', 'batter_player_id', 'outcome']]


In [7]:
# Merge the pitcher_df DataFrame with the batter_df DataFrame
pitcher_df = pitcher_df.merge(
    batter_df, on=['game_id', 'inning', 'batter_player_id'])


In [8]:
pitcher_df.head()

,game_id,pitcher_home_away,pitcher_team,pitcher_player_id,pitcher_player_name,starter,inning,batter_player_id,home_team,away_team,outcome
0,CHN201204050,0,CHN,dempr002,Ryan Dempster,1,1,desmi001,CHN,WAS,single
1,CHN201204050,0,CHN,dempr002,Ryan Dempster,1,1,espid001,CHN,WAS,walk
2,CHN201204050,0,CHN,dempr002,Ryan Dempster,1,1,zimmr001,CHN,WAS,out in play
3,CHN201204050,0,CHN,dempr002,Ryan Dempster,1,1,laroa001,CHN,WAS,strikeout
4,CHN201204050,0,CHN,dempr002,Ryan Dempster,1,1,wertj001,CHN,WAS,out in play


In [9]:
len(pitcher_df)

2035626

In [10]:
pitcher_df = pitcher_df.drop(columns=['inning', 'batter_player_id'])

In [ ]:
# Define the outcomes for the pitcher and whether they were a starter or relief pitcher
outcomes = ['out_in_play', 'strikeout', 'single', 'walk',
            'double', 'home_run', 'stolen_base', 'triple']

for outcome in outcomes:
    # Create a starting_pitcher column for each outcome
    pitcher_df[f'starting_pitcher_{outcome}'] = pitcher_df.apply(
        lambda row: 1 if row['outcome'] == outcome and row['starter'] == 1 else 0, axis=1)

    # Create a relief_pitcher column for each outcome
    pitcher_df[f'relief_pitcher_{outcome}'] = pitcher_df.apply(
        lambda row: 1 if row['outcome'] == outcome and row['starter'] == 0 else 0, axis=1)
